In [2]:
'''
Hello again everyone! This is Darshan S
Intern at CODSOFT, India.

I'm very thrilled to share my Second Task at CodSoft Internship September 2023 viz. CREDIT CARD FRAUD DETECTION MODEL.
More details on the structure, working and requirements are available in the README files of respective folder name corresponding to the Task Name.
'''

# AUTHOR: DARSHAN S
# TASK NAME: Credit Card Fraud Detection
# 2nd Task in the List of Tasks
# TASK CATEGORY: Machine Learning
# DATE OF SUBMISSION: 26 September 2023
# LinkedIn Profile: https://linkedin.com/in/arcticblue/
# GitHub Repository: https://github.com/azuregray/CodSoft-InternshipTasks/


# Lets start the code by importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import IncrementalPCA
from tqdm import tqdm

# Load training dataset
train_data = pd.read_csv("fraudTrain.csv")

# Load testing dataset
test_data = pd.read_csv("fraudTest.csv")

# Combine training and testing data for encoding consistency
combined_data = pd.concat([train_data, test_data], axis=0)

# Extract relevant features from the "trans_date_trans_time" column
def extract_datetime_features(df):
    df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
    df['day_of_week'] = df['trans_date_trans_time'].dt.dayofweek
    df['hour_of_day'] = df['trans_date_trans_time'].dt.hour
    df.drop('trans_date_trans_time', axis=1, inplace=True)
    return df

combined_data = extract_datetime_features(combined_data)

# Drop irrelevant columns (you can customize this based on your data)
columns_to_drop = ["first", "last", "job", "dob", "trans_num", "street"]
combined_data.drop(columns_to_drop, axis=1, inplace=True)

# Separate features and target variable
X_combined = combined_data.drop("is_fraud", axis=1)
y_combined = combined_data["is_fraud"]

# Encode the "merchant" and "category" columns using LabelEncoder
label_encoder = LabelEncoder()
X_combined["merchant"] = label_encoder.fit_transform(X_combined["merchant"])
X_combined["category"] = label_encoder.fit_transform(X_combined["category"])

# One-hot encode categorical variables with handle_unknown='ignore'
categorical_columns = ["gender", "city", "state"]
onehot_encoder = OneHotEncoder(sparse_output=False, drop="first", handle_unknown='ignore')

X_combined_categorical = onehot_encoder.fit_transform(X_combined[categorical_columns])

# Standardize the numeric features
scaler = StandardScaler()
X_combined_numeric = scaler.fit_transform(X_combined.drop(categorical_columns, axis=1))

# Combine one-hot encoded categorical features with numeric features
X_combined_encoded = np.hstack((X_combined_numeric, X_combined_categorical))

# Split the combined data back into training and test datasets
X_train = X_combined_encoded[:len(train_data)]
X_test = X_combined_encoded[len(train_data):]
y_train = y_combined[:len(train_data)]
y_test = y_combined[len(train_data):]

# Address class imbalance using SMOTE on the training data
sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

# Apply Incremental PCA for dimensionality reduction
n_components = 100  # Adjust the number of components as needed
ipca = IncrementalPCA(n_components=n_components)

# Apply Incremental PCA to training data with progress bar
for batch in tqdm(np.array_split(X_resampled, 10), desc="Applying Incremental PCA"):
    ipca.partial_fit(batch)

# Transform the training and testing data
X_resampled_pca = ipca.transform(X_resampled)
X_test_pca = ipca.transform(X_test)

# Define and train the Random Forest model
rf_classifier = RandomForestClassifier(random_state=42)

# Fit the model with the resampled data
rf_classifier.fit(X_resampled_pca, y_resampled)

# Predict using the trained model
y_pred = rf_classifier.predict(X_test_pca)

accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Display the necessary report on ML Model Metrics for evaluation
print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{confusion}")
print(f"Classification Report:\n{report}")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
Applying Incremental PCA: 100%|██████████| 10/10 [00:09<00:00,  1.05it/s]


Accuracy: 0.9964007198560288
Confusion Matrix:
[[4983    0]
 [  18    0]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4983
           1       0.00      0.00      0.00        18

    accuracy                           1.00      5001
   macro avg       0.50      0.50      0.50      5001
weighted avg       0.99      1.00      0.99      5001



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
